In [ ]:
%pip install -r requirements.txt

### MongoDB 연결


In [ ]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
from dotenv import load_dotenv
import os

# 환경 변수 로드
load_dotenv()

USERNAME = os.getenv("MONGO_USERNAME")
PASSWORD = os.getenv("MONGO_PASSWORD")
HOST = os.getenv("MONGO_HOST")
PORT = int(os.getenv("MONGO_PORT"))

url = f"mongodb://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/"

# MongoDB 연결
try:
    client = MongoClient(url)
    client.admin.command('ping')
    print("Successfully connected to MongoDB!")
except ConnectionFailure as e:
    print(f"MongoDB connection failed: {e}")

db = client['s307_db']
collection = db['s307_collection']


### PDF 재생성 설정


In [ ]:
from reportlab.pdfgen import canvas
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import os

# MongoDB의 root 객체에서 페이지 크기 정보 가져오기
root_data = collection.find_one({"page_num": 0})
page_width = root_data['width']
page_height = root_data['height']
page_count = root_data['page_count']
# 디버깅
print(f"PDF Info: {page_width} x {page_height}, Total Pages: {page_count}")


# 폰트 매핑: DB에 저장된 폰트명 -> 실제 윈도우 폰트 파일
# 중요 ⭐ 이거 폰트를 다운받아서 어떻게 해야할지 고민해야함
font_mapping = {
    'DotumChe': {
        'normal': 'C:/Windows/Fonts/gulim.ttc',
        'bold': 'C:/Windows/Fonts/malgunbd.ttf',  # 돋움체 볼드 대신 맑은 고딕 볼드 사용
    },
    'Dotum': {
        'normal': 'C:/Windows/Fonts/gulim.ttc',
        'bold': 'C:/Windows/Fonts/malgunbd.ttf',
    },
    'Malgun': {
        'normal': 'C:/Windows/Fonts/malgun.ttf',
        'bold': 'C:/Windows/Fonts/malgunbd.ttf',
    },
}

# 기본 폰트 정의 (Fallback용 - Helvetica는 한글 미지원하므로 한글 폰트 사용)
DEFAULT_FONT = 'MalgunGothic-Fallback'
pdfmetrics.registerFont(TTFont(DEFAULT_FONT, 'C:/Windows/Fonts/malgun.ttf'))

registered_fonts = {}  # 이미 등록된 폰트 추적


### 폰트 등록 함수


In [ ]:
def register_font_from_db(fontname):
    """DB의 폰트명을 분석하여 실제 폰트 파일로 등록"""
    # 이미 등록된 폰트면 재사용
    if not fontname or fontname in registered_fonts:
        # 폰트가 없거나 이미 등록된 경우, 한글 지원 폰트로 fallback
        return registered_fonts.get(fontname, DEFAULT_FONT)
    
    # DB 폰트명 분석 (예: "FAAAAH+DotumChe,Bold" -> base="DotumChe", is_bold=True)
    parts = fontname.split('+')[-1]  # 접두어 제거
    # + 앞에 붙어있는 FAAAH같은 것들은 무시. Subset Font로 용량을 줄이기 위해 임베딩 된 폰트 식별자임
    is_bold = 'Bold' in parts or 'bold' in parts
    base_font = parts.split(',')[0]  # 베이스 폰트명 추출
    
    # 매핑된 폰트 파일 찾기
    font_file = None
    for key in font_mapping:
        if key in base_font:
            if is_bold and 'bold' in font_mapping[key]:
                font_file = font_mapping[key]['bold']
            else:
                font_file = font_mapping[key]['normal']
            break
    
    # 폰트 파일 등록
    if font_file and os.path.exists(font_file):
        try:
            pdfmetrics.registerFont(TTFont(fontname, font_file))
            registered_fonts[fontname] = fontname
            return fontname
        except:
            # 폰트 등록 실패 시 한글 지원 폰트로 fallback
            registered_fonts[fontname] = DEFAULT_FONT
            return DEFAULT_FONT
    else:
        # 폰트 파일을 찾지 못한 경우 한글 지원 폰트로 fallback
        registered_fonts[fontname] = DEFAULT_FONT
        return DEFAULT_FONT


### MongoDB 데이터로 PDF 재생성


In [ ]:
def reconstruct_pdf_from_mongodb(output_filename="output.pdf"):
    """MongoDB에 저장된 문자 정보로 PDF 재생성"""
    
    # PDF Canvas 생성
    c = canvas.Canvas(output_filename, pagesize=(page_width, page_height))
    
    # DB에서 페이지별로 데이터 가져오기 (page_num > 0인 문서만, 정렬)
    pages_data = collection.find({"page_num": {"$gt": 0}}).sort("page_num", 1)
    
    for page_data in pages_data:
        chars = page_data.get('chars', [])
        
        # 각 문자를 원래 위치에 배치
        for char in chars:
            text = char['text']
            x0 = char['x0']
            # y0값이 생김으로 인해 top -> y0으로 변경
            y0 = char['y0']
            
            # 폰트 정보 가져오기
            fontname = char.get('fontname', None)
            font_size = char.get('size', 10)
            
            # 폰트 등록 및 사용
            if fontname:
                font_to_use = register_font_from_db(fontname)
            else:
                font_to_use = DEFAULT_FONT
            

            ## 혹시 모를 색상이 칠해져있는 글씨를 위해 추가
            # 색상 정보 가져오기
            non_stroking_color = char.get('non_stroking_color', None)
            stroking_color = char.get('stroking_color', None)
            
            # 텍스트 그리기
            try:
                c.setFont(font_to_use, font_size)
                
                # Canvas를 활용한 색깔 설정
                # 텍스트 색상 설정 (non_stroking_color = 텍스트 내부 색상)
                if non_stroking_color is not None:
                    if isinstance(non_stroking_color, (tuple, list)):
                        if len(non_stroking_color) == 3:
                            c.setFillColorRGB(*non_stroking_color)
                        elif len(non_stroking_color) == 1:
                            c.setFillGray(non_stroking_color[0])
                    elif isinstance(non_stroking_color, (int, float)):
                        c.setFillGray(non_stroking_color)
                
                # 텍스트 외곽선 색상 설정
                    if isinstance(stroking_color, (tuple, list)):
                        if len(stroking_color) == 3:
                            c.setStrokeColorRGB(*stroking_color)
                        elif len(stroking_color) == 1:
                            c.setStrokeGray(stroking_color[0])
                    elif isinstance(stroking_color, (int, float)):
                        c.setStrokeGray(stroking_color)
                
                c.drawString(x0, y0, text)
            except:
                continue  # 렌더링 실패 시 건너뛰기
        
        # 다음 페이지로
        c.showPage()
    
    # PDF 저장
    c.save()

# PDF 재생성 실행
reconstruct_pdf_from_mongodb("reconstructed.pdf")


### 연결 종료


In [ ]:
client.close()
